# Environment Setup
Includes package installation for sequence synthesis. Will take a few minutes.


## Magenta setup

In [1]:
import glob

BASE_DIR = "gs://download.magenta.tensorflow.org/models/music_vae/colab2"

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library


print('Importing libraries and defining some helper functions...')
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

print('Done')

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 1.4MB 3.2MB/s 
     |████████████████████████████████| 3.6MB 21.8MB/s 
     |████████████████████████████████| 5.6MB 23.5MB/s 
     |████████████████████████████████| 92kB 9.3MB/s 
     |████████████████████████████████| 1.5MB 34.1MB/s 
     |██████████████████████████

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


Instructions for updating:
non-resource variables are not supported in the long term
Done


# 2-Bar Drums Model

Below are 4 pre-trained models to experiment with. The first 3 map the 61 MIDI drum "pitches" to a reduced set of 9 classes (bass, snare, closed hi-hat, open hi-hat, low tom, mid tom, high tom, crash cymbal, ride cymbal) for a simplified but less expressive output space. The last model uses a [NADE](http://homepages.inf.ed.ac.uk/imurray2/pub/11nade/) to represent all possible MIDI drum "pitches".

* **drums_2bar_oh_lokl**: This *low* KL model was trained for more *realistic* sampling. The output is a one-hot encoding of 2^9 combinations of hits. It has a single-layer bidirectional LSTM encoder with 512 nodes in each direction, a 2-layer LSTM decoder with 256 nodes in each layer, and a Z with 256 dimensions. During training it was given 0 free bits, and had a fixed beta value of 0.8. After 300k steps, the final accuracy is 0.73 and KL divergence is 11 bits.
* **drums_2bar_oh_hikl**: This *high* KL model was trained for *better reconstruction and interpolation*. The output is a one-hot encoding of 2^9 combinations of hits. It has a single-layer bidirectional LSTM encoder with 512 nodes in each direction, a 2-layer LSTM decoder with 256 nodes in each layer, and a Z with 256 dimensions. During training it was given 96 free bits and had a fixed beta value of 0.2. It was trained with scheduled sampling with an inverse sigmoid schedule and a rate of 1000. After 300k, steps the final accuracy is 0.97 and KL divergence is 107 bits.
* **drums_2bar_nade_reduced**: This model outputs a multi-label "pianoroll" with 9 classes. It has a single-layer bidirectional LSTM encoder with 512 nodes in each direction, a 2-layer LSTM-NADE decoder with 512 nodes in each layer and 9-dimensional NADE with 128 hidden units, and a Z with 256 dimensions. During training it was given 96 free bits and has a fixed beta value of 0.2. It was trained with scheduled sampling with an inverse sigmoid schedule and a rate of 1000. After 300k steps, the final accuracy is 0.98 and KL divergence is 110 bits.
* **drums_2bar_nade_full**:  The output is a multi-label "pianoroll" with 61 classes. A single-layer bidirectional LSTM encoder with 512 nodes in each direction, a 2-layer LSTM-NADE decoder with 512 nodes in each layer and 61-dimensional NADE with 128 hidden units, and a Z with 256 dimensions. During training it was given 0 free bits and has a fixed beta value of 0.2. It was trained with scheduled sampling with an inverse sigmoid schedule and a rate of 1000. After 300k steps, the final accuracy is 0.90 and KL divergence is 116 bits.

#Load Pretrained Models


In [2]:
drums_models = {}
# One-hot encoded.
drums_config = configs.CONFIG_MAP['cat-drums_2bar_small']
drums_models['drums_2bar_oh_lokl'] = TrainedModel(drums_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/drums_2bar_small.lokl.ckpt')
drums_models['drums_2bar_oh_hikl'] = TrainedModel(drums_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/drums_2bar_small.hikl.ckpt')

# Multi-label NADE.
drums_nade_reduced_config = configs.CONFIG_MAP['nade-drums_2bar_reduced']
drums_models['drums_2bar_nade_reduced'] = TrainedModel(drums_nade_reduced_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/drums_2bar_nade.reduced.ckpt')
drums_nade_full_config = configs.CONFIG_MAP['nade-drums_2bar_full']
drums_models['drums_2bar_nade_full'] = TrainedModel(drums_nade_full_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/drums_2bar_nade.full.ckpt')


INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]

INFO:tensorflow:
Decoder Cells:
  units: [256, 256]

Instructions for updating:
Use `tf.cast` instead.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
INFO:tensorflow:Restoring parameters from gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/drums_2bar_small.lokl.ckpt
INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256

# Download & Extract midi (1-time only)

In [3]:
!rm -r midifiles

os.mkdir('midifiles')
!wget --no-check-certificate -r "https://ollaregang.pythonanywhere.com/static/MIDI/AllDrums.zip" -O "midifiles/Input.zip" 

rm: cannot remove 'midifiles': No such file or directory
will be placed in the single file you specified.

--2021-06-10 10:07:39--  https://ollaregang.pythonanywhere.com/static/MIDI/AllDrums.zip
Resolving ollaregang.pythonanywhere.com (ollaregang.pythonanywhere.com)... 35.173.69.207
Connecting to ollaregang.pythonanywhere.com (ollaregang.pythonanywhere.com)|35.173.69.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7188 (7.0K) [application/zip]
Saving to: ‘midifiles/Input.zip’

midifiles/Input.zip 100%[===================>]   7.02K  --.-KB/s    in 0s      

2021-06-10 10:07:39 (1001 MB/s) - ‘midifiles/Input.zip’ saved [7188/7188]

FINISHED --2021-06-10 10:07:39--
Total wall clock time: 0.4s
Downloaded: 1 files, 7.0K in 0s (1001 MB/s)


In [4]:
from zipfile import ZipFile

zipname = "midifiles/Input.zip"
with ZipFile(zipname,'r') as zpfile:
  files = zpfile.namelist()
  for f in files:
    if (f.endswith('.mid')):
      zpfile.extract(f, 'midifiles')

# Generate Interpolations

## Load midi files

In [5]:
input_drums_midi_data = [
    tf.io.gfile.GFile(fn, mode='rb').read()
    for fn in sorted(tf.io.gfile.glob('./midifiles/MIDI_2bar/*.mid'))]

## Convert midi to needed format

In [ ]:
!rm -r midifiles/converted_MIDI
!rm -r midifiles/interpolation_sequences

rm: cannot remove 'midifiles/converted_MIDI': No such file or directory
rm: cannot remove 'midifiles/interpolation_sequences': No such file or directory


In [6]:
from note_seq.protobuf import music_pb2
import copy
import note_seq

alphabet = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"]

os.mkdir('midifiles/converted_MIDI')
os.mkdir('midifiles/interpolation_sequences')

drums_input_seqs =music_pb2.NoteSequence();
def set_to_drums(ns):
  for n in ns.notes:
    n.instrument=9
    n.is_drum = True
  return ns


def midi_to_drum_note(input_drums_midi_data):
  drums = []
  for m in input_drums_midi_data:
    drum = mm.midi_to_note_sequence(m)
    set_to_drums(drum)
    drums.append(drum)
  return drums

drums_input_seqs = midi_to_drum_note(input_drums_midi_data)

for index in range(len(drums_input_seqs)):
  midi_path = "midifiles/converted_MIDI/converted_" + alphabet[index] + ".mid"
  mm.note_sequence_to_midi_file(drums_input_seqs[index], midi_path)

In [7]:
input_drums_midi_data = [
    tf.io.gfile.GFile(fn, mode='rb').read()
    for fn in sorted(tf.io.gfile.glob('./midifiles/converted_MIDI/*.mid'))]

##Load & Listen beats

In [8]:
drums_input_seqs = [mm.midi_to_sequence_proto(m) for m in input_drums_midi_data]

extracted_beats = [];
for ns in drums_input_seqs:

  test=drums_nade_full_config.data_converter.from_tensors(
      drums_nade_full_config.data_converter.to_tensors(ns)[1])
  extracted_beats.extend(test)
  #print(test)
for index, beat in enumerate(extracted_beats):
  print("Beat", index)
  play(beat)

Beat 0


Beat 1


Beat 2


Beat 3


Beat 4


Beat 5


Beat 6


Beat 7


Beat 8


Beat 9


## Interpolation

In [9]:
drums_interp_model = "drums_2bar_oh_hikl" 

# temperature = 0.8 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
temperature = 0.5
num_steps = 8 

for index1, beat1 in enumerate(extracted_beats):
  for index2, beat2 in enumerate(extracted_beats):
    start_beat = extracted_beats[index1]
    end_beat = extracted_beats[index2]
    drums_interp = interpolate(drums_models[drums_interp_model],
                              start_beat, 
                              end_beat, 
                              num_steps=num_steps, 
                              temperature=temperature)
    mm.sequence_proto_to_midi_file(drums_interp, "midifiles/interpolation_sequences/interpolation_" + alphabet[index1] + alphabet[index2] + ".mid")


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/trained_model.py:384: RuntimeWarning: invalid value encountered in double_scalars
  return np.sin((1.0-t)*omega) / so * p0 + np.sin(t*omega)/so * p1
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/trained_model.py:382: RuntimeWarning: invalid value encountered in arccos
  np.squeeze(p1/np.linalg.norm(p1))))


## Concatenate midi

In [10]:
os.mkdir('midifiles/temp_concatenations')
os.mkdir('midifiles/final_concatenations')

In [11]:
from mido import MidiFile
from mido import MidiTrack
from mido import MetaMessage


def concatenate_beats0(path1, path2, resolution1, resolution2, save_path):

  mid0 = MidiFile()
  mid0.ticks_per_beat=resolution1
  track0 = MidiTrack()
  mid0.tracks.append(track0)

  mid = MidiFile(path1)

  for msg in mid.tracks[2]:
      mid0.tracks[0].append(msg)

  mid2 = MidiFile(path2)

  for msg in mid2.tracks[2][:-1]:
      msg.time=int(msg.time*resolution1/resolution2)
      mid0.tracks[0].append(msg)

  mid0.save(save_path)

In [12]:
def concatenate_beats1(path1, path2, resolution1, resolution2, save_path):

  mid0 = MidiFile()
  mid0.ticks_per_beat=resolution1
  track0 = MidiTrack()
  mid0.tracks.append(track0)

  mid = MidiFile(path1)

  for msg in mid.tracks[0]:
      mid0.tracks[0].append(msg)

  mid2 = MidiFile(path2)

  for msg in mid2.tracks[2][:-1]:
      msg.time=int(msg.time*resolution1/resolution2)
      mid0.tracks[0].append(msg)

  mid0.save(save_path)

In [13]:
def concatenate_beats2(path1, path2, resolution1, resolution2, save_path):

  mid0 = MidiFile()
  mid0.ticks_per_beat=resolution1
  track0 = MidiTrack()
  mid0.tracks.append(track0)

  mid = MidiFile(path1)
  for msg in mid.tracks[0]:
      mid0.tracks[0].append(msg)

  mid2 = MidiFile(path2)

  for msg in mid2.tracks[0][:-1]:
      msg.time=int(msg.time*resolution1/resolution2)
      mid0.tracks[0].append(msg)

  mid0.save(save_path)

##Start concatenations

In [14]:
for letter in alphabet:
  concatenate_beats0("/content/midifiles/converted_MIDI/converted_"+ letter + ".mid",
                    "/content/midifiles/converted_MIDI/converted_"+ letter + ".mid",
                    96,
                    96,
                    '/content/midifiles/temp_concatenations/longer_midi_' + letter + '.mid')

In [15]:
for letter1 in alphabet:
  for letter2 in alphabet:
    if letter1 != letter2:
      concatenate_beats1("/content/midifiles/temp_concatenations/longer_midi_"+ letter1 + ".mid",
                        "/content/midifiles/interpolation_sequences/interpolation_" + letter1 + letter2 + ".mid",
                        96,220,
                        "/content/midifiles/temp_concatenations/intermediate1_" + letter1 + letter2 + ".mid")
      
      concatenate_beats2("/content/midifiles/temp_concatenations/intermediate1_"+ letter1 + letter2 + ".mid",
                        "/content/midifiles/temp_concatenations/longer_midi_"+ letter2 + ".mid",
                        96,96,
                        "/content/midifiles/temp_concatenations/intermediate2_" + letter1 + letter2 + ".mid")
      
      concatenate_beats1("/content/midifiles/temp_concatenations/intermediate2_"+ letter1 + letter2 + ".mid",
                        "/content/midifiles/interpolation_sequences/interpolation_" + letter2 + letter1 + ".mid",
                        96,220,
                        "/content/midifiles/final_concatenations/interpolated_beat_" + letter1 + letter2 + ".mid")
      

# Upload to server

In [ ]:
import requests

for letter1 in alphabet:
  for letter2 in alphabet:
    if letter1 != letter2:
      url = 'http://OLLAREGANG.pythonanywhere.com/interpolationUploader/interpolation_' + letter1 + letter2 + ".mid"
      midiToUpload = open("/content/midifiles/final_concatenations/interpolated_beat_" + letter1 + letter2 + ".mid", 'rb')
      x = requests.post(url, data = midiToUpload)
      print(x.text)